In [9]:
!git clone https://bitbucket.org/jadslim/german-traffic-signs.git

Cloning into 'german-traffic-signs'...
Updating files: 100% (4/4)
Updating files: 100% (4/4), done.


In [15]:
!ls german-traffic-signs

# the "signnames" contains the traffic signs' labels

# and the rest of 3 files are "pickled" files:
# Pickle in Python is primarily used in serializing and deserializing a Python object structure. 
# In other words, it's the process of converting a Python object into a byte stream to store it in a file/database, 
# maintain program state across sessions, or transport data over the network

# now we need the UNPICKLE them to get the image datas

signnames.csv
test.p
train.p
valid.p


In [11]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import random

In [12]:
np.random.seed(0)